## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [294]:
import numpy as np
import random
SIZE = 4
weights = np.array([[3,4,5,3],[12,13,2,14],[13,9,3,6],[1,7,11,9], [3,4,5,3],[12,13,2,14],[13,9,3,6],[1,7,11,9]])

def weight_in_population(population):
    weight = 0
    row = 0
    for column in population:
        weight += weights[row][column - 1]
        row += 1
    return weight

#returns true if 2 queens collide otherwise false
def check_column(population):
    numbers = {}
    for number in population:
        numbers[number] = 1
        
    return len(numbers.keys()) == SIZE

def check_for_collision(population):
    if check_column(population) == False:
        return True
    
    size = len(population)
    
    #print(population)
    for index in range(size):
        for row in range(size):
            if row == index:
                continue
            if np.abs(row - index) == np.abs(population[row] - population[index]):
                return True
        
    return False
            
def generate_table(chromosome_length: int):
    used = {}
    table = np.array([x for x in range(chromosome_length)])
    np.random.shuffle(table)
    
    return table

def initialize_population(population_size, chromosome_length):
    population = np.array([generate_table(chromosome_length) for x in range(population_size)])
    
    return population

# POPULATION_SIZE = 10
# CHROMOSOME_LENGTH = 4
# population = initialize_population(POPULATION_SIZE, CHROMOSOME_LENGTH)

def evaluate_fitness(population):
    fitness_scores = []
    for individual in population:
        fitness = weight_in_population(individual)
        are_colliding = check_for_collision(individual)
        mean = np.mean(weights) * SIZE;
        #print(weight)
        if are_colliding == True:
            fitness -= mean
            if fitness > 0:
                fitness_scores.append(fitness)
            else:
                fitness_scores.append(0)
        else:
            fitness_scores.append(fitness)
        
    return fitness_scores
    
#a = evaluate_fitness(population)
#print(a)
    
def select_parents(population, fitness_scores):
    #roulette selection
    #print(fitness_scores)
    probabilities = fitness_scores / np.sum(fitness_scores) 
    return random.choices(population, weights = probabilities, k = 2)
    
def crossover(parents):
    p1 = parents[0]
    p2 = parents[1]
    
   # print(p1,p2)
    offspring = []
    
    len_p1 = len(p1)
    len_p2 = len(p2)
    
    child1 = p1[:len_p1//2], p2[len_p2//2:]
    child2 = p1[:len_p1//2], p2[:len_p1//2]
    
    child1 = np.concatenate((child1[0],child1[1]))
    child2 = np.concatenate((child2[0],child2[1]))

    offspring = np.array([child1, child2])
    return offspring

def mutate(chromosome, mutation_rate):
    alpha = random.random()
    if alpha < mutation_rate:
        chromosome = (int)(chromosome + alpha * 10) % SIZE
        
    return chromosome

def genetic_algorithm(population_size, chromosome_length, generations, mutation_rate):    
    # initialize the population
    population = initialize_population(population_size, chromosome_length )
    for _ in range(generations):
        # Fitness evaluation
        
        fitness_scores = evaluate_fitness(population)
        # Selection
        parents = select_parents(population, fitness_scores)
        
        #print(parents)
        # Crossover
        offsprings = crossover(parents)
        #print(offsprings)
        # Mutation
        #print(offsprings)
        mutated_offspring = np.array([np.array([mutate(child, mutation_rate) for child in offspring]) for offspring in offsprings])
            
        fitness = evaluate_fitness(mutated_offspring) 
            
        population = np.append(population,np.array([mutated_offspring[0]]), axis = 0)
        population = np.append(population,np.array([mutated_offspring[1]]), axis = 0)
        
        fitness_scores.append(fitness[0])
        fitness_scores.append(fitness[1])
        
        probabilities = fitness_scores / np.sum(fitness_scores) 

        population = random.choices(population, weights = probabilities, k = population_size)

    return population

# Test the complete genetic algorithm
population_size = 10
chromosome_length = SIZE
generations = 100
mutation_rate = 0.1

final_population = genetic_algorithm(population_size, chromosome_length, generations, mutation_rate)
#print(final_population)

max_fitness = -1;
best_population = []

for pop in final_population:
    fitness = evaluate_fitness([pop])
    if fitness[0] > max_fitness:
        max_fitness = fitness[0]
        best_population = pop
        
print(max_fitness)
print(best_population)

for row in range(len(best_population)):
    line = ""
    for column in range(len(best_population)):
        if best_population[row] == column:
            line += " Q "
        else:
            line += ' - '
    print(line)

22
[2 0 3 1]
 -  -  Q  - 
 Q  -  -  - 
 -  -  -  Q 
 -  Q  -  - 
